## Projeto de Lógica de Programação II

* Ler uma lista de objetos de um JSON;

* Realizar um mapeamento, um filtro e uma redução;

* Permitir que os dados possam ser lidos individualmente, atualizados e deletados (manter JSON atualizado);

* Garantir que todas as operações tenham validações (try-except, raise);

* Criar uma função para obter uma lista de tuplas, com o máximo (ou o mínimo) valor de algum atributo numérico, com o primeira posição contendo o nome do elemento e a segunda o valor máximo (o motivo de ser lista é porque pode ter mais de um elemento com valor máximo);

* Esta função deve ter um parâmetro opcional, que diz qual das estatísticas você deseja obter (mínimo ou máximo);

* Obter algum dado estatístico simples, como média, máximo e mínimo (exemplo, a média de idade dos professores que dão aula de exatas, caso o conjunto de dados seja do tema professores);

* Salvar dados estatísticos em um CSV.

In [1]:
# Importa as bibliotecas

import json
import csv
import time
import copy


In [2]:
# Carregar dados do arquivo JSON
def carregar_dados(arquivo_json):
    try:
        with open(arquivo_json, 'r') as arquivo:
            dados = json.load(arquivo)
        return dados
    except FileNotFoundError:
        print(f"O arquivo {arquivo_json} não foi encontrado. Criando um novo arquivo.")
        return []
        
# Salvar dados em um arquivo JSON
def salvar_arquivo(dados: list[dict], arquivo_json: str) -> bool:
    try:
        with open(arquivo_json, 'w') as arquivo:
            arquivo.write(json.dumps(dados))
            return True
    except Exception:
        return False
    


In [3]:
# Função para adicionar uma nova pessoa
def criar_pessoa(dados: list[dict], arquivo_json: str) -> bool:
    try:
        pessoa = {
            'nome': obter_nome(),
            'sobrenome': obter_sobrenome(),
            'idade': obter_idade(),
            'sexo': obter_sexo(),
            'altura': obter_altura(),
            'salario': obter_salario()
        }

        dados.append(pessoa)
        if salvar_arquivo(dados, arquivo_json):
            return True

    except ValueError as e:
        pass

    return False

# Funções para coletar os dados de uma nova pessoa
def obter_nome() -> str:
    return input('Informe o nome:').title()

def obter_sobrenome() -> str:
    return input('Informe o sobrenome:').title()

def obter_idade() -> int:
    while True:
        try:
            idade = int(input('Informe a idade: '))
            if 18 <= idade <= 120:
                return idade
            else:
                print("A idade deve estar entre 18 e 120 anos.")
        except ValueError:
            print("Por favor, digite um número inteiro válido.") 

def obter_sexo() -> str:
    while True:
        sexo = input('Informe o sexo (F ou M): ').capitalize()
        if sexo in ['F', 'M']:
            return sexo
        else:
            print("Por favor, digite 'Feminino' ou 'Masculino'.")

def obter_altura() -> float:
    while True:
        try:
            altura = float(input('Informe a altura (m): '))
            if altura > 0:
                return altura
            else:
                print("A altura deve ser maior que zero.")
        except ValueError:
            print("Entrada inválida. Por favor, digite um valor numérico.")

def obter_salario() -> float:
    while True:
        try:
            salario = float(input('Informe o salário: '))
            if salario > 0:
                return salario
            else:
                print("O salário deve ser maior que zero.")
        except ValueError:
            print("Entrada inválida. Por favor, digite um valor numérico.")

In [4]:
# Função para buscar uma pessoa espeficica
def buscar_pessoa(dados: list[dict], nome: str, sobrenome: str) -> list[dict]:
    encontradas = [pessoa for pessoa in dados if pessoa['nome'] == nome and pessoa['sobrenome'] == sobrenome]
    if not encontradas:
        print(f"Pessoa não encontrada: {nome} {sobrenome}")
    return encontradas

# Função para atualizar os dados de uma pessoa
def atualizar_pessoa(dados: list[dict], arquivo_json: str) -> bool:
    nome = obter_nome()
    sobrenome = obter_sobrenome()
    pessoas_encontradas = buscar_pessoa(dados, nome, sobrenome)

    if not pessoas_encontradas:
        return False  
    
    if len(pessoas_encontradas) == 1:
        pessoa_encontrada = pessoas_encontradas[0]

        pessoa_encontrada.update({
            'idade': obter_idade(),
            'sexo': obter_sexo(),
            'altura': obter_altura(),
            'salario': obter_salario()
        })

        if salvar_arquivo(dados, arquivo_json):
            return True  
        else:
            return False  
    else:
        print("Mais de um registro encontrado para: ",nome,sobrenome)
        return False  

In [5]:
# Função para deletar os dados de uma pessoa
def deletar_pessoa(dados: list[dict], arquivo_json: str) -> bool:
    nome = obter_nome()
    sobrenome = obter_sobrenome()
    pessoas_encontradas = buscar_pessoa(dados, nome, sobrenome)
    
    if not pessoas_encontradas:
        return False  

    if len(pessoas_encontradas) > 1:
        print("Mais de um registro encontrado para: ",nome,sobrenome)
        return False 

    pessoa_encontrada = pessoas_encontradas[0]
    dados.remove(pessoa_encontrada)

    if salvar_arquivo(dados, arquivo_json):
        return True  
    else:
        return False 

In [6]:
# Função para exibir os dados
def exibir_dados(pessoas: list[dict]) -> str:
    if not pessoas:
        return "Nenhuma pessoa para exibir."

    resultados = []
    for pessoa in pessoas:
        resultado_pessoa = '\n'.join([f"{chave.capitalize()}: {valor}" for chave, valor in pessoa.items()])
        resultados.append(resultado_pessoa)

    return '\n\n'.join(resultados)

# Função para exibir os dados de uma pessoa especifica
def exibir_pessoa(dados: list[dict], arquivo_json: str) -> bool:
    nome = obter_nome()
    sobrenome = obter_sobrenome()
    pessoas_encontradas = buscar_pessoa(dados, nome, sobrenome)
    print(exibir_dados(pessoas_encontradas))

In [7]:
# Função para obter o salário máximo e mínimo
def salario_max_min(dados, maximo=True):
    if not dados:
        return "Não há dados a serem exibidos"
    lista_pessoas = [(pessoa["nome"], pessoa["salario"]) for pessoa in dados]
    max_salario = max(lista_pessoas, key=lambda x: x[1]) if maximo else min(lista_pessoas, key=lambda x: x[1])
    return max_salario

# Função para obter a quantidade de pessoas pelo sexo
def pessoas_sexo(dados, sexo):
    return len(list(filter(lambda pessoa: pessoa["sexo"] == sexo, dados)))
    

# Função para calcular a média de algum campo especifico (altura/salário)
def calcular_media(dados, campo):
    if not dados:
        return 0  # Retorna 0 se a lista de dados estiver vazia
    total_valores = 0
    for pessoa in dados:
        total_valores += pessoa[campo]

    media = total_valores / len(dados)
    media_arredondada = round(media, 2)
    return media_arredondada

In [8]:
# Função para gerar estatísticas
def gerar_estatisticas(dados):
    pessoas_f = pessoas_sexo(dados, 'F')
    pessoas_m = pessoas_sexo(dados, 'M')
    max_nome, max_salario = salario_max_min(dados, maximo=True)
    min_nome, min_salario = salario_max_min(dados, maximo=False)
    media_altura = calcular_media(dados, 'altura')
    media_salario = calcular_media(dados, 'salario')

    estatisticas = {
        "pessoas_f": pessoas_f,
        "pessoas_m": pessoas_m,
        "max_salario_nome": max_nome,
        "max_salario": max_salario,
        "min_salario_nome": min_nome,
        "min_salario": min_salario,
        "media_altura": media_altura,
        "media_salario": media_salario
    }

    return estatisticas

In [13]:
# Função para gerar arquivo CSV das estatísticas
def gerar_csv(estatisticas):
    # Verificar se o arquivo CSV já existe
    try:
        with open('estatisticas.csv', 'r', newline='', encoding='utf-8') as csvfile:
            reader = csv.reader(csvfile)
            headers = next(reader)  # Lê os cabeçalhos, se existirem
            arquivo_existente = True
    except FileNotFoundError:
        arquivo_existente = False

    # Adicionar dados ao arquivo CSV
    with open('estatisticas.csv', 'a', newline='', encoding='utf-8') as csvfile:
        fieldnames = [
            "pessoas_f",
            "pessoas_m",
            "max_salario_nome",
            "max_salario",
            "min_salario_nome",
            "min_salario",
            "media_altura",
            "media_salario"
        ]

        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # Se o arquivo não existir, escreva os cabeçalhos
        if not arquivo_existente:
            writer.writeheader()

        # Escreva os dados
        writer.writerow(estatisticas)

In [10]:

# Função menu de opções, usuario escolhe as opções de 0 a 9, se 0 encerra o programa, se diferente desses, o programa pede para inserir um valor válido.
def menu_de_opções(dados: list[dict], arquivo_json: str):

    while True:
        # time.sleep tem como finalidade estabelecer um intervalo temporaral de 0,5 segundos na exibição das opções do menu.
        print('1-Criar pessoa')
        time.sleep(0.5)
        print('2-Atualizar pessoa')
        time.sleep(0.5)
        print('3-Deletar pessoa')
        time.sleep(0.5)
        print('4-Exibir pessoa')
        time.sleep(0.5)
        print('5-Gerar estatísticas;')
        time.sleep(0.5)
        print('6-Gerar CSV das estatísticas;')
        time.sleep(0.5)
        print('0-Encerrar programa.')
        time.sleep(0.5)

        opçao = input('\nO que deseja fazer? ')

        if opçao == '0':
          print('Programa encerrado!')
          break

        elif opçao == '1':
          if criar_pessoa(dados, arquivo_json):
              print("Registro salvo com sucesso!")
          else:
              print("Falha ao salvar o registro. Verifique os dados e tente novamente.")

        elif opçao == '2':
          if atualizar_pessoa(dados, arquivo_json):
              print("Registro salvo com sucesso!")
          else:
              print("Falha ao atualizar o registro. Verifique os dados e tente novamente.")          

        elif opçao == '3':
          if deletar_pessoa(dados, arquivo_json):
              print("Registro excluido com sucesso!")
          else:
              print("Falha ao excluir o registro. Verifique os dados e tente novamente.") 
        elif opçao == '4':
          print(exibir_pessoa(dados,arquivo_json))

        elif opçao == '5':
              estatisticas = gerar_estatisticas(dados)
              print("Estatísticas geradas:", estatisticas)

        elif opçao == '6':
          estatisticas = gerar_estatisticas(dados)
          gerar_csv(estatisticas)
          print("Arquivo CSV gerado.")

        else:
            print('Opção invalida!')



In [14]:
arquivo_json = 'dados_pessoa.json'
dados = carregar_dados(arquivo_json)
if dados is not None:
    menu_de_opções(dados,arquivo_json)
else:
    # Trate o caso em que o arquivo não foi encontrado ou houve um erro de decodificação JSON
    print("Falha ao carregar dados.")  

1-Criar pessoa
2-Atualizar pessoa
3-Deletar pessoa
4-Exibir pessoa
5-Gerar estatísticas;
6-Gerar CSV das estatísticas;
0-Encerrar programa.
Estatísticas geradas: {'pessoas_f': 468, 'pessoas_m': 535, 'max_salario_nome': 'Carlos', 'max_salario': 9957.84, 'min_salario_nome': 'Isabel', 'min_salario': 0, 'media_altura': 1.75, 'media_salario': 3451.32}
1-Criar pessoa
2-Atualizar pessoa
3-Deletar pessoa
4-Exibir pessoa
5-Gerar estatísticas;
6-Gerar CSV das estatísticas;
0-Encerrar programa.
Arquivo CSV gerado.
1-Criar pessoa
2-Atualizar pessoa
3-Deletar pessoa
4-Exibir pessoa
5-Gerar estatísticas;
6-Gerar CSV das estatísticas;
0-Encerrar programa.
Estatísticas geradas: {'pessoas_f': 468, 'pessoas_m': 535, 'max_salario_nome': 'Carlos', 'max_salario': 9957.84, 'min_salario_nome': 'Isabel', 'min_salario': 0, 'media_altura': 1.75, 'media_salario': 3451.32}
1-Criar pessoa
2-Atualizar pessoa
3-Deletar pessoa
4-Exibir pessoa
5-Gerar estatísticas;
6-Gerar CSV das estatísticas;
0-Encerrar programa.
Ar